# Transformaciones avanzadas en dataset de e-commerce

## Datos base

In [1]:
# importar librerías
import pandas as pd
import numpy as np

# Clientes
clientes = pd.DataFrame({
    'cliente_id': range(1, 6),
    'nombre': ['Ana', 'Juan', 'María', 'Pedro', 'Laura'],
    'segmento': ['Premium', 'Regular', 'Premium', 'Regular', 'VIP']
})

# Pedidos
pedidos = pd.DataFrame({
    'pedido_id': range(1, 11),
    'cliente_id': np.random.choice(range(1, 6), 10),
    'producto': np.random.choice(['A', 'B', 'C', 'D'], 10),
    'precio': np.random.uniform(50, 500, 10).round(2),
    'fecha': pd.date_range('2024-01-01', periods=10)
})

print("Clientes y pedidos cargados")

Clientes y pedidos cargados


## Enriquecer datos con joins

In [2]:
# Unir pedidos con información de clientes
pedidos_enriquecidos = pd.merge(
    pedidos, 
    clientes, 
    on='cliente_id', 
    how='left'
)

print("Pedidos con información de clientes:")
print(pedidos_enriquecidos.head())

Pedidos con información de clientes:
   pedido_id  cliente_id producto  precio      fecha nombre segmento
0          1           2        D  276.00 2024-01-01   Juan  Regular
1          2           4        C   81.71 2024-01-02  Pedro  Regular
2          3           2        C  466.47 2024-01-03   Juan  Regular
3          4           3        C  232.90 2024-01-04  María  Premium
4          5           3        C  206.34 2024-01-05  María  Premium


## Calcular métricas derivadas

In [3]:
# Calcular métricas por cliente
metricas_cliente = pedidos_enriquecidos.groupby(['cliente_id', 'nombre', 'segmento']).agg({
    'pedido_id': 'count',
    'precio': ['sum', 'mean', 'max'],
    'fecha': 'max'  # Última compra
}).round(2)

# Aplanar columnas multi-nivel
metricas_cliente.columns = ['num_pedidos', 'total_gastado', 'gasto_promedio', 'gasto_maximo', 'ultima_compra']
metricas_cliente = metricas_cliente.reset_index()

print("\nMétricas por cliente:")
print(metricas_cliente)


Métricas por cliente:
   cliente_id nombre segmento  num_pedidos  total_gastado  gasto_promedio  \
0           2   Juan  Regular            3        1210.75          403.58   
1           3  María  Premium            2         439.24          219.62   
2           4  Pedro  Regular            2         382.61          191.30   
3           5  Laura      VIP            3         537.15          179.05   

   gasto_maximo ultima_compra  
0        468.28    2024-01-08  
1        232.90    2024-01-05  
2        300.90    2024-01-09  
3        338.22    2024-01-10  


## Validar reglas de negocio

In [4]:
def validar_reglas_negocio(df):
    validaciones = []
    
    # VIP deben tener al menos 2 pedidos
    vip_insuficientes = df[(df['segmento'] == 'VIP') & (df['num_pedidos'] < 2)]
    if len(vip_insuficientes) > 0:
        validaciones.append(f"VIPs con pocos pedidos: {len(vip_insuficientes)}")
    
    # Premium no deben exceder gasto máximo
    premium_excesivos = df[(df['segmento'] == 'Premium') & (df['gasto_maximo'] > 800)]
    if len(premium_excesivos) > 0:
        validaciones.append(f"Premiums con gastos excesivos: {len(premium_excesivos)}")
    
    return validaciones

reglas_incumplidas = validar_reglas_negocio(metricas_cliente)
print(f"\nReglas de negocio incumplidas: {reglas_incumplidas}")


Reglas de negocio incumplidas: []


## Verificación
¿Qué tipo de join usarías cuando quieres mantener todos los registros de una tabla principal?
- Usaría un left join, porque garantiza que todos los registros de la tabla principal se mantengan en el resultado, incluso si no existe correspondencia en la tabla secundaria. Esto es clave en análisis de datos y pipelines productivos, ya que evita perder información y permite detectar problemas de integridad referencial cuando aparecen valores nulos tras el join.

¿Cómo decides qué métricas calcular para un análisis específico?
- Las métricas se definen según el objetivo del análisis y las decisiones de negocio que se quieren apoyar. A partir de ese objetivo se identifica que comportamientos son relevantes (frecuencia, valor, recencia, variabilidad) y luego se traducen esos conceptos en agregaciones. Además, se considera la granularidad del análisis y que las métricas sean accionables, interpretables y coherentes con los datos disponibles.